The results of background subtraction are non-zero pixel (i.e. foreground) coordinates. These can saved as coordinate arrays. A coordinate array really is just a Darr ragged array (see separate library). This makes it easy to read the data in other environments, e.g. R. But also Birdwatcher provides functionality to access coordinate arrays, and look at the results in videos and plots.

In [ ]:
import numpy as np
import birdwatcher as bw
import birdwatcher.movementdetection as md

from birdwatcher.plotting import imshow_frame
import matplotlib.pyplot as plt
%matplotlib inline

### Create a video object

In [ ]:
vfs = bw.VideoFileStream(r'..\videos\zf20s_low.mp4')

### Create coordinate arrays

In the previous notebook, we showed how to use background subtraction and save the results into a coordinate array step-by-step. In this notebook, we will do movement detection the easy way, by using a high-level function, with the results saved as coordinate arrays.

We will use the default settings, meaning that the default values for the background subtractor MOG2 and some pre- and postprocessing steps are automatically taken care of. For the example video this will work great, but for other videos you might need more control of the various settings. Have a look at the next notebooks to find out how to modify the default settings of this function.

In [ ]:
coords, coordscount, coordsmean = md.detect_movement(vfs, bgs_type=bw.BackgroundSubtractorMOG2,
                                                     analysispath='output/', ignore_firstnframes=50, 
                                                     overwrite=True)

This high-level function returns three arrays, which are disk-based Darr arrays. They can be very large and hence not fit in RAM. The coordinate arrays are saved within a 'movement' folder with the name of the VideoFileStream.

### Accessing coordinate arrays

The coordinate array can be accessed in other python sessions like so:

In [ ]:
coords = bw.CoordinateArrays('output/movement_zf20s_low/coords.darr')

In [ ]:
coords.metadata

To access coordscount and coordsmean just run:

In [ ]:
coordscount = coords.get_coordcount()
coordsmean = coords.get_coordmean()

### Look at `coords`

'Coords' provide the detected foreground pixels in a ragged array. Therefore, you will have the coordinates of all 'movement pixels' per frame. You can have a look at the results as a video in a separate window: (press 'q' to quit)

In [ ]:
coords.show(framerate=vfs.avgframerate)

Let's look at the results of frame 131:

In [ ]:
coords[131]

You can see a simple numpy array with the x, y coordinates of all 'movement pixels'.

We can view the results as a black and white image by the `get_frame` method, which returns a frame instead of coordinates.

In [ ]:
frame = coords.get_frame(131)
imshow_frame(frame)

Let's look at the original frame as comparison:

In [ ]:
imshow_frame(vfs.get_frame(131))

We can create a video of the results as well:

In [ ]:
coords.tovideo('output/zf20s_coords.mp4', framerate=vfs.avgframerate)

Or a selection of the results by indicating start and end frame numbers:

In [ ]:
coords.tovideo('output/zf20s_coords_selection.mp4', startframe=100, endframe=200, framerate=vfs.avgframerate)

If you want to do more things before saving to video, just use `iter_frames` which turns it into a Frames object with many more methods. Make sure you use three color channels and set coordinates to value 255 if you want them white.

In [ ]:
frames = (coords.iter_frames(startframe=100, endframe=200, nchannels=3, value=255)
          .draw_framenumbers()
          .tovideo('output/zf20s_coords_selection_framenumbers.mp4', framerate=vfs.avgframerate))

### Look at `coordscount`

The 'coordscount' shows the number of pixels that belong to the foreground, e.g. 'movement pixels', per frame. Thus, higher peaks means more movement.

In [ ]:
plt.plot(coordscount)
plt.title('number of pixels above treshold')
plt.xlabel('frame number')
plt.ylabel('number of pixels')

### Look at `coordsmean`

The 'coordsmean' shows the mean coordinates per frame. This could be used to look at the location of the subject during the video. The blue line shows the horizontal coordinates (left-rigth) and the orange line show the vertical coordinates (top-bottom).

In [ ]:
plt.plot(coordsmean)
plt.title('coordinates of pixels above treshold')
plt.xlabel('frame number')
plt.ylabel('pixel coordinate')
plt.legend(['left-right', 'top-bottom'])

We can also look at the mean coordinates in a video using the original frames + the mean coordinate per frame superimposed on it as a circle:

In [ ]:
vfs_circle = (vfs.iter_frames()
              .draw_framenumbers()
              .draw_circles(coordsmean)
              .tovideo('output/zf20s_coords_center.mp4', framerate=vfs.avgframerate))

It's also possible to change the settings of the circle, such as the radius and color:

In [ ]:
vfs_circle = (vfs.iter_frames()
              .draw_framenumbers()
              .draw_circles(coordsmean, radius=50, color=(0, 100, 255))
              .tovideo('output/zf20s_coords_center_orange.mp4', framerate=vfs.avgframerate))

There is also a high-level function that creates a similar video, but better! It will produce a video of the original one with coordinate results ánd the mean results superimposed.

In [ ]:
vfs_results = md.create_movementvideo(vfs, coords, videofilepath='output/movementvideoexample.mp4')